In [46]:
import pandas as pd

# Pseudocritical values for different gases in Rankine (psi)
pseudocritical_values = {
    'methane': {'pressure': 673.8, 'temperature': 343.0},
    'ethane': {'pressure': 706.5, 'temperature': 549.7},
    'propane': {'pressure': 616.3, 'temperature': 617.7},
    'ibutane': {'pressure': 527.9, 'temperature': 693.1},
    'ipentane': {'pressure': 488.6, 'temperature': 735.0},
    'hexane': {'pressure': 435.9, 'temperature': 658.0},
    'heptane': {'pressure': 395.4, 'temperature': 789.6},
    'helium': {'pressure': 506.5, 'temperature': 87.9},
    'oxygen': {'pressure': 5048.4, 'temperature': 154.6},
    'H2S': {'pressure': 1306.0, 'temperature': 672.4},
    'CO2': {'pressure': 1071.0, 'temperature': 547.6},
}

# Read the gas composition data from a CSV file into a DataFrame
file_path = 'newest_dataframe_model.csv'  
selected_features = ['T (K)', 'P (MPa)', 'methane', 'ethane', 'propane', 'ibutane', 'ipentane',
                     'hexane', 'heptane', 'helium', 'oxygen', 'H2S', 'CO2']
df = pd.read_csv(file_path)

# Convert mole fractions from percentages to fractions
for col in selected_features[2:]:
    df[col] = df[col] / 100.0

def calculatee_pseudocritical_properties(row):
    total_pressure = 0.0
    total_temperature = 0.0
    total_H2S = 0.0
    total_CO2 = 0.0

    for gas, fraction in row.items():
        if gas in pseudocritical_values:
            pc_pressure = pseudocritical_values[gas]['pressure']
            pc_temperature = pseudocritical_values[gas]['temperature']
            total_pressure += fraction * pc_pressure
            total_temperature += fraction * pc_temperature
            if gas == 'H2S':
                total_H2S += fraction
            elif gas == 'CO2':
                total_CO2 += fraction
        else:
            pass

    A = total_H2S + total_CO2
    epsilon = 120 * (A ** 0.9 - A ** 1.6) + 15 * (total_H2S ** 0.5 - total_H2S ** 4.0)

    corrected_Tpc = total_temperature - epsilon
    corrected_Ppc = total_pressure * (total_temperature / corrected_Tpc)
    Ppc = total_pressure 
    Tpc = total_temperature

    return Ppc, Tpc
df['Pseudocritical Pressure (psi)'], df['Pseudocritical Temperature (Rankine)'] = zip(*df.apply(calculatee_pseudocritical_properties, axis=1))


def calculate_pseudocritical_properties(row):
    total_pressure = 0.0
    total_temperature = 0.0
    total_H2S = 0.0
    total_CO2 = 0.0

    for gas, fraction in row.items():
        if gas in pseudocritical_values:
            pc_pressure = pseudocritical_values[gas]['pressure']
            pc_temperature = pseudocritical_values[gas]['temperature']
            total_pressure += fraction * pc_pressure
            total_temperature += fraction * pc_temperature
            if gas == 'H2S':
                total_H2S += fraction
            elif gas == 'CO2':
                total_CO2 += fraction
        else:
            pass

    A = total_H2S + total_CO2
    epsilon = 120 * (A ** 0.9 - A ** 1.6) + 15 * (total_H2S ** 0.5 - total_H2S ** 4.0)

    corrected_Tpc = total_temperature - epsilon
    corrected_Ppc = total_pressure * (total_temperature / corrected_Tpc)
    

    return corrected_Tpc, corrected_Ppc


# Calculate corrected pseudocritical pressure and temperature for each row in the DataFrame
df['Corrected Pseudocritical Pressure (psi)'], df['Corrected Pseudocritical Temperature (Rankine)'] = zip(*df.apply(calculatee_pseudocritical_properties, axis=1))

# df['Ppc'] = 
# df['Tpc'] = 

# df. to_csv("dataframeeee_corr.csv", index=False)

In [47]:

def calculate_pseudoreduced_properties(row):
    Tc = row['Corrected Pseudocritical Temperature (Rankine)']
    Pc = row['Corrected Pseudocritical Pressure (psi)']
    T = row['T (K)'] * 1.8  # Convert temperature from Kelvin to Rankine
    P = row['P (MPa)'] * 145.037738  # Convert pressure from MPa to psi

    Tpr = T / Tc
    Ppr = P / Pc

    return Tpr, Ppr

# Calculate pseudoreduced temperature and pressure for each gas in the DataFrame
df['Pseudoreduced Temperature'] = 0.0
df['Pseudoreduced Pressure'] = 0.0

for index, row in df.iterrows():
    Tpr, Ppr = calculate_pseudoreduced_properties(row)
    df.at[index, 'Pseudoreduced Temperature'] = Tpr
    df.at[index, 'Pseudoreduced Pressure'] = Ppr

print(df)


      Gas #    T (K)  d (mol/l)  P (MPa)  cv (J/mol-K)  cp (J/mol-K)  \
0       2.0  165.933       1.00  1.17898       27.0032       42.1156   
1       2.0  169.184       2.00  2.04973       29.8695       59.0063   
2       2.0  176.240       3.00  2.80779       32.3094       84.2345   
3       2.0  181.681       4.00  3.42183       34.5254      126.0547   
4       2.0  185.480       5.00  3.89052       36.7615      204.0512   
...     ...      ...        ...      ...           ...           ...   
3673  193.0  246.745      19.04  7.12054       37.4728      101.7986   
3674  193.0  238.458      20.23  6.49203       36.6412       91.9511   
3675  193.0  228.957      21.42  5.81102       35.9184       84.5390   
3676  193.0  218.239      22.61  5.09979       35.1920       78.8956   
3677  193.0  206.362      23.80  4.37882       34.2480       74.4535   

       w (m/s)  P (MPa).1  cv (J/mol-K).1  cp (J/mol-K).1  ...  water  oxygen  \
0     308.9646     1.1792         26.4849         41.4

In [48]:
import pandas as pd
import gascompressibility as gc

# Load your dataset
data = df.copy()

# Function to calculate pseudoreduced properties using gascompressibility library
def calculate_pseudoreduced_properties(row):
    Tc = row['Corrected Pseudocritical Temperature (Rankine)']
    Pc = row['Corrected Pseudocritical Pressure (psi)']
    T = row['T (K)'] * 1.8  # Convert temperature from Kelvin to Rankine
    P = row['P (MPa)'] * 145.037738  # Convert pressure from MPa to psi

    Tpr = T / Tc
    Ppr = P / Pc

    return Tpr, Ppr

# Calculate pseudoreduced temperature and pressure for each gas in the DataFrame
data['Pseudoreduced Temperature'] = 0.0
data['Pseudoreduced Pressure'] = 0.0
data['z'] = 0.0
data['Corrected Tpr'] = 0.0
data['Corrected Ppr'] = 0.0

for index, row in data.iterrows():
    Tpr, Ppr = calculate_pseudoreduced_properties(row)
    try:
        z_factor = gc.calc_z(Pr=Ppr, Tr=Tpr)
        data.at[index, 'Pseudoreduced Temperature'] = Tpr
        data.at[index, 'Pseudoreduced Pressure'] = Ppr
        data.at[index, 'z'] = z_factor

        # Store corrected and original values of Tpr and Ppr
        data.at[index, 'Corrected Tpr'] = Tpr
        data.at[index, 'Corrected Ppr'] = Ppr
    except RuntimeError as e:
        print(f"Convergence issue at index {index}: {e}")

# Filter rows based on Z Factor values within the specified range
filtered_data = data[(data['z'] >= 0.55) & (data['z'] <= 1.85)]

# Display the filtered DataFrame
print(filtered_data)


Convergence issue at index 1694: Failed to converge
      Gas #    T (K)  d (mol/l)  P (MPa)  cv (J/mol-K)  cp (J/mol-K)  \
0       2.0  165.933       1.00  1.17898       27.0032       42.1156   
1       2.0  169.184       2.00  2.04973       29.8695       59.0063   
2       2.0  176.240       3.00  2.80779       32.3094       84.2345   
17      2.0  176.767      18.00  2.97205       31.1300      100.7543   
18      2.0  171.908      19.00  2.51695       30.7187       86.8249   
...     ...      ...        ...      ...           ...           ...   
3658  193.0  256.961       1.19  2.17538       30.1797       46.5998   
3659  193.0  264.036       2.38  3.87818       33.7609       63.4514   
3660  193.0  270.867       3.57  5.28131       36.5508       84.6583   
3661  193.0  275.345       4.76  6.39975       38.8038      111.9564   
3662  193.0  277.967       5.95  7.26683       40.6603      145.7121   

       w (m/s)  P (MPa).1  cv (J/mol-K).1  cp (J/mol-K).1  ...  hydrogen  \
0     3

In [49]:
df.describe()

,Gas #,T (K),d (mol/l),P (MPa),cv (J/mol-K),cp (J/mol-K),w (m/s),P (MPa).1,cv (J/mol-K).1,cp (J/mol-K).1,...,water,oxygen,argon,hydrogen,Pseudocritical Pressure (psi),Pseudocritical Temperature (Rankine),Corrected Pseudocritical Pressure (psi),Corrected Pseudocritical Temperature (Rankine),Pseudoreduced Temperature,Pseudoreduced Pressure
count,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,...,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000
mean,100.266177,237.454010,8.656552,6.526990,37.435273,165.898171,376.096709,7.399638,-3.304391,123.060945,...,0.000122,0.000025,0.001623,0.004654,662.464732,358.768401,662.464732,358.768401,1.210892,1.450343
std,54.342142,40.254612,5.028223,2.692213,46.145108,456.532776,108.289572,10.608425,434.596470,954.837573,...,0.000990,0.000207,0.018595,0.025479,57.913402,35.069505,57.913402,35.069505,0.348920,0.662139
min,2.000000,128.258000,0.230000,0.321100,25.343100,35.849000,130.840800,-89.509700,-18951.835500,-39254.520200,...,0.000000,0.000000,0.000000,0.000000,239.068659,122.643519,239.068659,122.643519,0.675976,0.076413
25%,54.000000,204.361500,4.350000,4.799035,31.442375,62.168925,317.776575,4.993650,27.762900,51.908775,...,0.000000,0.000000,0.000000,0.000000,652.199844,347.277080,652.199844,347.277080,1.026758,1.044179
50%,101.000000,231.106000,8.460000,6.230515,34.216150,95.058200,357.677000,6.881150,31.072550,80.679050,...,0.000000,0.000000,0.000000,0.000000,668.411866,356.911850,668.411866,356.911850,1.153437,1.355490
75%,147.000000,262.914000,12.600000,8.210595,36.724850,141.706125,392.634575,8.708550,34.564450,123.818200,...,0.000000,0.000000,0.000000,0.000000,673.953952,370.278830,673.953952,370.278830,1.321792,1.809381
max,193.000000,377.808000,23.800000,17.484210,1446.217000,10571.151200,1099.176500,470.119900,527.984000,22420.466600,...,0.012000,0.002830,0.252000,0.210500,939.139240,481.090328,939.139240,481.090328,5.544968,6.079455


In [50]:
filtered_data.to_csv('df_for_models.csv', index=False)